# Tests d'estimation et p-valeurs

## Estimation du maximum de vraisemblance, p-valeur globale

Si on est sûr du modèle de probabilité en $\cos^2(\alpha-\theta)$,
mais qu'on ne connaît pas $\theta$, on peut chercher la valeur de
$\theta$ qui donne le maximum de vraisemblance, qui peut se calculer
en fonction des probabilités théoriques et empiriques pour l'ensemble
des $\alpha$.


In [3]:
import numpy as np
import pandas as pd # type: ignore
import scipy.stats as sps
from scipy.stats import chi2

# On va travailler sur le jeu de données '1photon-polar-30-intervalles'.
# Remarquer que les angles de mesure sont réguliers, c'est ce qui va nous
# permettre de calculer des valeurs-p ci-dessous. (Sinon il faudrait binner.)
data = pd.read_csv('datasets/jour1-1-facile/1photon-polar-30-intervalles.csv', sep=';')
obs = data.X
table_alpha = data.alpha

# Définir une fonction pour la loi de probabilité, en fonction des angles en degrés.
# La fonction peut opérer sur des nombres ou des tableaux.
def f_proba(alpha, theta):
    return np.cos(np.radians(alpha - theta)) ** 2

# Définir une fonction de vraisemblance. (Spécifique aux valeurs de table_alpha.)
def L(theta):
    table_probas = f_proba(table_alpha, theta)
    return (np.sum(np.log(table_probas[obs == 1]))
           + np.sum(np.log(1 - table_probas[obs == 0]))
           )

# Repérer l'angle ayant le maximum de vraisemblance à 0.1 degré près (overkill).
test_theta = np.arange(0, 180, 0.1)
indice_MLE = np.argmax(np.asarray([L(theta) for theta in test_theta]))
theta_MLE = test_theta[indice_MLE]
print(f'{theta_MLE=}')

/usr/lib/python3/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


theta_MLE=29.900000000000002


### p-valeur et intervalle de confiance

Quand on fait l'hypothèse qu'une certaine valeur de $\theta$ est correcte,
alors pour chaque valeur de $\alpha$, on peut calculer une p-valeur en
comparant la probabilité empirique de détection pour cet $\alpha$ à la loi
en $\cos^2(\alpha-\theta)$ ; et on peut ensuite combiner l'ensemble de ces
p-valeurs en une seule, spécifique à la valeur testée de $\theta$.

Cette p-valeur est alors la probabilité que l'hypothèse sur la valeur de
$\theta$ soit correcte.  Si elle est trop faible (p. ex. inférieure à 0.05),
on rejette l'hypothèse.  Ceci permet alors de calculer un intervalle de
confiance qui sera l'ensemble des valeurs de $\theta$ que l'on ne peut pas
écarter, parce que la p-valeur est supérieure au seuil choisi.


In [ ]:
# Calcul des valeurs-p pour chaque angle de mesure rencontré,
# à partir du nombre de fois où on l'a rencontré et de la valeur
# moyenne de l'observation en X.
table_alpha_uniques = np.unique(table_alpha)
nb_alpha = np.asarray([len(table_alpha[table_alpha == a]) for a in table_alpha_uniques])
moyenne_obs_alpha = np.asarray([np.mean(obs[table_alpha == a]) for a in table_alpha_uniques])
#t = sps.norm.ppf(0.05) # FIXME this was in the old lab, not used?

# Le bloc suivant élimine les angles pour lesquels les probabilités
# sont trop hautes ou trop basses.  Essayez sans pour voir ;
# pourquoi posent-elles problème ?
indices_alpha_ok = (moyenne_obs_alpha > 0.1) & (moyenne_obs_alpha < 0.9)
table_alpha_uniques = table_alpha_uniques[indices_alpha_ok]
moyenne_obs_alpha = moyenne_obs_alpha[indices_alpha_ok]
nb_alpha = nb_alpha[indices_alpha_ok]

# Fonction pour calculer la valeur-p pour un theta spécifique.
def pvalue(theta):
    # Probabilité attendue pour chaque angle.
    p0 = f_proba(table_alpha_uniques, theta)

    # Valeur-p pour chaque angle.
    pvalues = [2*(1 - sps.norm.cdf(abs(p - x)*(n/(p*(1-p)))**0.5))
               for p,x,n in zip(moyenne_obs_alpha, p0, nb_alpha)]

    # Combinaison des valeurs-p par la méthode de Fisher.
    Tchi = -2 * sum([np.log(x) for x in pvalues])

    # Retrouver la valeur-p correspondante via la distribution chi²
    # avec 2*(nombre de valeurs-p) degrés de liberté.
    return 1 - sps.chi2.cdf(Tchi, 2*len(table_alpha_uniques))

# Intervalle de confiance.
table_pvalues = np.asarray([pvalue(theta) for theta in test_theta])
print(table_pvalues[table_pvalues > 0.05])
indices_pvaleur_grande = np.where(table_pvalues > 0.05)[0]
CI = [test_theta[indices_pvaleur_grande[0]], test_theta[indices_pvaleur_grande[-1]]]
CI

[0.05780228 0.08048041 0.10539839 0.12055046 0.12171962 0.11701056
 0.10699797 0.09291783 0.07645766 0.05511524]


/tmp/ipykernel_1852548/1043220646.py:27: RuntimeWarning: divide by zero encountered in log
  Tchi = -2 * sum([np.log(x) for x in pvalues])


[29.0, 29.900000000000002]

## Inégalité CHSH, test d'intrication

L'inégalité de Clauser-Horne-Shimony-Holt (CHSH) montre que, si l'expérience de mesure à 2 photons pouvait être modélisée par une théorie à variables cachées locales, on aurait : $|S| \leqslant 2$, où :
$$S = E(a1,b1) - E(a1,b2) + E(a2,b1) + E(a2,b2),$$
avec $a1,b1$ et $a1,b2$ et $a2,b1$ et $a2,b2$ 4 combinaisons différentes des angles de mesure $\alpha$ et $\beta$,
et :
$$E(x,y) = P_{11}^{xy} - P_{10}^{xy} - P_{01}^{xy} + P_{00}^{xy}  $$
avec $ P_{11}^{xy} = \mathbb{P}(Xa = 1,Xb=1|\alpha = x, \beta = y)$, etc.

On peut générer $\hat{S}$ la valeur empirique de $S$. Soit $n$ le nombre de points de mesure pour chaque combinaison d'angles $\alpha, \beta$ (supposé le même pour toutes les combinaisons ; sinon voir plus bas). Soit $X_{i}^{ab} \in \{0,1\}$ et $Y_{i}^{ab}\in \{0,1\}$ le résultat de la $i$-ème mesure avec les angles $a,b$. On définit alors :
$$I_i(a,b) = X_{i}^{ab} Y_{i}^{ab} + X_{i}^{ab} (1-Y_{i}^{ab}) - (1-X_{i}^{ab})Y_{i}^{ab} + (1-X_{i}^{ab})(1-X_{i}^{ab}).$$
On peut alors écrire la variable aléatoire :
$$Z_i = I(a1,b1) - I(a1,b2) + I(a2,b1) + I(a2,b2)$$
où l'on voit que les $Z$ sont des variables aléatoires IID telles que : $Z_i \in [-2,2]$ et $\mathbb{E}[Z_i] = S$.
On peut alors utiliser l'inégalité de Hoeffding pour borner la p-valeur : cette inégalité entraîne que, pour des variables IID $X_{i,...,n}$ bornées dans un intervalle $[A,B]$ :
$$\mathbb{P}\left(|\frac{1}{n}\sum X_i - \mathbb{E}[X] |> t\right) \leqslant 2\exp\left(\frac{-nt^2}{(A-B)^2}\right)$$

Dans notre cas, la probabilité d'observer une statistique au moins aussi extrême que $\hat S : |\hat S| >2$, en supposant qu'on est dans une théorie à variables cachées locales, a la borne supérieure :
$$pval = 2\exp\left(\frac{-n(\hat S - 2)^2}{4^2}\right).$$

L'inégalité CHSH est valable pour toutes les valeurs des angles $a1, b1, a2, b2$. En pratique, pour détecter une violation de CHSH dans une expérience, on pourra choisir des valeurs qu'on sait correspondre à un maximum de $|S|$, par exemple $a1 = 0°, a2 = 45°, b1 = 22.5°, b2 = 67.5°$ pour l'état EPR.  En revanche, si on ne connaît pas les angles ou l'état des paires de photons, on peut commencer par identifier la combinaison d'angles la plus prometteuse, en cherchant les angles qui maximisent $|\hat{S}|$ sur une petite fraction des données. On peut alors faire le test pour ces angles-là en utilisant les données restantes (afin de ne pas invalider le test). (On pourra de nouveau utiliser la fonction `train_test_split` de `sklearn`.)

In [ ]:
import math
import numpy as np
import pandas as pd # type: ignore
from sklearn.model_selection import train_test_split

df_xx = pd.read_csv('datasets/jour2-1-facile/2photons-xx-1M.csv', sep=';')
df_epr = pd.read_csv('datasets/jour2-1-facile/2photons-epr-1M.csv', sep=';')

# Fonction de comptage des mesures et des coïncidences (XX, XY, ...)
# ainsi que la quantité E, dans des tableaux 2D indexés par les angles
# (alpha sur l'axe 0, beta sur l'axe 1).
def count_detections(data, angles):
    nb_angles = len(angles)
    (count_measurements, count_xx, count_xy, count_yx, count_yy) = (
        np.zeros((nb_angles, nb_angles)) for _ in range(5)
    )
    for i, alpha in enumerate(angles):
        for j, beta in enumerate(angles):
            select_angles = (data.alpha == alpha) & (data.beta == beta)
            count_measurements[i, j] = select_angles.sum()
            (Xa, Xb, Ya, Yb) = (data[name][select_angles] == 1 for name in ('Xa', 'Xb', 'Ya', 'Yb'))
            count_xx[i, j] = (Xa & Xb).sum()
            count_xy[i, j] = (Xa & Yb).sum()
            count_yx[i, j] = (Ya & Xb).sum()
            count_yy[i, j] = (Ya & Yb).sum()

    E = (count_xx + count_yy - count_yx - count_xy) / count_measurements
    return (count_measurements, count_xx, count_xy, count_yx, count_yy, E)

# Fonction de calcul de S à partir du tableau E.
def S_from_E(E):
    n = E.shape[0]
    assert E.shape == (n, n), f'Le tableau E doit être carré, pas de forme {E.shape}'
    S = (E.reshape(n, 1, n, 1)   # alpha, beta
         + E.reshape(1, n, n, 1) # alpha', beta
         - E.reshape(n, 1, 1, n) # alpha, beta'
         + E.reshape(1, n, 1, n) # alpha', beta'
        )
    return S

# Pour le jeu de données testé, recherche des angles où |S| est max
# sur une partie des données.
data = df_epr
angles = np.unique(data[['alpha', 'beta']])
nb_angles = len(angles)
data_search_S, data_test_S = train_test_split(data, train_size = 0.25)
E = count_detections(data_search_S, angles)[-1]
absS = np.abs(S_from_E(E))
S_max = absS[np.isfinite(absS)].max() # Éviter les valeurs 0/0 avec isfinite().
(i_alpha1, i_alpha2, i_beta1, i_beta2) = np.argwhere(absS == S_max)[0]
(alpha1, alpha2, beta1, beta2) = (angles[i] for i in (i_alpha1, i_alpha2, i_beta1, i_beta2))
print(alpha1, alpha2, beta1, beta2)

# Calcul d'une borne de la p-valeur de |S| sur le reste des données
# pour les angles trouvés où |S| était max ci-dessus.  Pour le nombre
# de mesures, au cas où ce ne serait pas le même pour les différentes
# combinaisons d'angles, vu qu'on calcule une borne, on peut prendre
# le min.
(counts, _, _, _, _, E) = count_detections(data_test_S, angles)
S = E[i_alpha1, i_beta1] + E[i_alpha2, i_beta1] - E[i_alpha1, i_beta2] + E[i_alpha2, i_beta2]
num = min(counts[i_alpha1, i_beta1], counts[i_alpha2, i_beta1],
          counts[i_alpha1, i_beta2], counts[i_alpha2, i_beta2])
pval = 2 * math.exp(-((abs(S) - 2)**2 * num) / 16)
print(f'|S| max = {abs(S)}, pvalue <= {pval}')


123.75 78.75 11.25 146.25
|S| max = 2.781900644533751, pvalue <= 1.1493492073266981e-48


### Influence du nombre de points de mesure

Refaites le test avec les jeux de données de `datasets-jour3-supplement.zip` et commentez.